In [ ]:
!pip install numpy pandas


In [ ]:
import csv
import os.path
from taxUtils import *
from transaction import *
import tokens

In [ ]:
# Define the data directory
data_directory = "../data"
# copy /data/input to /data/account_main and then edit
user_data = "account_main"
raw_data = "raw"

# Load the user wallets
user_wallets = read_json_file(os.path.join(data_directory, user_data, "user_wallets.json"))
user_first_wallet = list(user_wallets.keys())[0]
print(user_first_wallet)
print(user_wallets)

# Load the user baker wallets
baker_wallets = read_json_file(os.path.join(data_directory, user_data, "baker_wallets.json"))

# Load the exchange wallets
exchange_wallets = read_json_file(os.path.join(data_directory, user_data, "exchange_wallets.json"))

In [ ]:
# This raw data can be downloaded once. Remove data/raw to clear the cache.

raw_delegations_path = os.path.join(data_directory, raw_data, "raw_delegations.json")
raw_transactions_path = os.path.join(data_directory, raw_data, "raw_transactions.json")
raw_originations_path = os.path.join(data_directory, raw_data, "raw_originations.json")
raw_reveals_path = os.path.join(data_directory, raw_data, "raw_reveals.json")
user_mints_path = os.path.join(data_directory, raw_data, "user_mints.json")
user_swaps_path = os.path.join(data_directory, raw_data, "user_swaps.json")
user_collects_path = os.path.join(data_directory, raw_data, "user_collects.json")
user_auction_bids_path = os.path.join(data_directory, raw_data, "user_auction_bids.json")
user_art_sales_path = os.path.join(data_directory, raw_data, "user_art_sales.json")
user_collection_sales_path = os.path.join(data_directory, raw_data, "user_collection_sales.json")
objktcom_collections_path = os.path.join(data_directory, raw_data, "objktcom_collections.json")
fa12_tokens_path = os.path.join(data_directory, raw_data, "fa12_tokens.json")
fa2_tokens_path = os.path.join(data_directory, raw_data, "fa2_tokens.json")

p = os.path.dirname(raw_reveals_path)
if os.path.exists(p) == False:
    os.makedirs(p)

# Get the user raw transactions and originations information
raw_reveals = None
if os.path.exists(raw_reveals_path):
    raw_reveals = read_json_file(raw_reveals_path)
    
if raw_reveals == None:
    raw_delegations = get_user_delegations(user_wallets)
    save_json_file(raw_delegations_path, raw_delegations)
    raw_transactions = get_user_transactions(user_wallets)
    save_json_file(raw_transactions_path, raw_transactions)
    raw_originations = get_user_originations(user_wallets)
    save_json_file(raw_originations_path, raw_originations)
    raw_reveals = get_user_reveals(user_wallets)
    save_json_file(raw_reveals_path, raw_reveals)

    # Get the user mints, swaps, collects, auction bids, art sales and collection sales
    user_mints = get_user_mints(user_wallets)
    user_swaps = get_user_swaps(user_wallets)
    user_collects = get_user_collects(user_wallets)
    user_auction_bids = get_user_auction_bids(user_wallets)
    user_art_sales = get_user_art_sales(user_wallets)
    user_collection_sales = get_user_collection_sales(user_wallets)

    save_json_file(user_mints_path, user_mints)
    save_json_file(user_swaps_path, user_swaps)
    save_json_file(user_collects_path, user_collects)
    save_json_file(user_auction_bids_path, user_auction_bids)
    save_json_file(user_art_sales_path, user_art_sales)
    save_json_file(user_collection_sales_path, user_collection_sales)

    # Get the list of FA2 contracts associated to the objkt.com collections
    objktcom_collections = get_objktcom_collections()

    # Get the list of FA1.2 and FA2 tokens
    fa12_tokens = get_fa12_tokens()
    fa2_tokens = get_fa2_tokens()

    save_json_file(objktcom_collections_path, objktcom_collections)
    save_json_file(fa12_tokens_path, fa12_tokens)
    save_json_file(fa2_tokens_path, fa2_tokens)
else:
    raw_delegations = read_json_file(raw_delegations_path)
    raw_transactions = read_json_file(raw_transactions_path)
    raw_originations = read_json_file(raw_originations_path)
    
    user_mints = read_json_file(user_mints_path)
    user_swaps = read_json_file(user_swaps_path)
    user_collects = read_json_file(user_collects_path)
    user_auction_bids = read_json_file(user_auction_bids_path)
    user_art_sales = read_json_file(user_art_sales_path)
    user_collection_sales = read_json_file(user_collection_sales_path)
    objktcom_collections = read_json_file(objktcom_collections_path)
    fa12_tokens = read_json_file(fa12_tokens_path)
    fa2_tokens = read_json_file(fa2_tokens_path)

In [ ]:
raw_transactions[1]['target']['address']

In [ ]:
# Get the main tezos tokens and smart contract aliases
token_aliases = {address: name for name, address in TOKENS.items()}
smart_contract_aliases = {address: alias for alias, address in SMART_CONTRACTS.items()}

# Define the burn addresses
burn_addresses = ["tz1burnburnburnburnburnburnburjAYjjX"]

# Process the raw transactions
transactions = []
unprocessed_transactions = []

for t in raw_transactions:
    has_params = "parameter" in t and isinstance(t['parameter'], dict)
    # Save the most relevant information
    transaction = {
        "timestamp": t["timestamp"],
        "level": t["level"],
        "kind": None,
        "entrypoint": t["parameter"]["entrypoint"] if has_params else None,
        "parameters": t["parameter"]["value"] if has_params else None,
        "initiator": t["initiator"]["address"] if "initiator" in t and isinstance(t['initiator'], dict) else None,
        "sender": t["sender"]["address"] if "sender" in t and isinstance(t['sender'], dict) else None,
        "target": t["target"]["address"] if "target" in t and isinstance(t['target'], dict) else None,
        "applied": t["status"] == "applied",
        "internal": False,
        "ignore": False,
        "mint": False,
        "collect": False,
        "active_offer": False,
        "art_sale": False,
        "collection_sale": False,
        "staking": False,
        "origination": False,
        "reveal": False,
        "delegation": False,
        "prize": False,
        "donation": False,
        "buy_tez": t["sender"]["address"] in exchange_wallets,
        "sell_tez": t["target"]["address"] in exchange_wallets,
        "amount": t["amount"] / 1e6,
        "fees": ((t["bakerFee"] + t["storageFee"] + t["allocationFee"]) / 1e6) if (t["status"] == "applied") else (t["bakerFee"] / 1e6),
        "tez_to_euros": t["quote"]["eur"],
        "tez_to_usd": t["quote"]["usd"],
        "token_id": None,
        "token_editions": None,
        "token_address": None,
        "hash": t["hash"],
        "comment": ""}

    # Add the information from the known user transactions
    hash = t["hash"]
    
    transaction = clean_transaction(transaction, hash, user_wallets, user_mints, user_swaps, user_collects, user_auction_bids, user_art_sales, user_collection_sales, baker_wallets, objktcom_collections)

    transaction = get_transaction_aliases(transaction, token_aliases, user_wallets, burn_addresses, objktcom_collections, fa12_tokens, fa2_tokens)

    # Save the unprocessed raw transactions
    if transaction["kind"] is None:
        unprocessed_transactions.append(t)

    # Add the processed transaction
    transactions.append(transaction)

# Process the raw originations
originations = []

for o in raw_originations:
    # Save the most relevant information
    origination = {
        "timestamp": o["timestamp"],
        "level": o["level"],
        "kind": "contract origination",
        "entrypoint": None,
        "parameters": None,
        "initiator": o["initiator"]["address"] if "initiator" in o else None,
        "sender": o["sender"]["address"],
        "target": None,
        "applied": o["status"] == "applied",
        "internal": False,
        "ignore": False,
        "mint": False,
        "collect": False,
        "active_offer": False,
        "art_sale": False,
        "collection_sale": False,
        "staking": False,
        "origination": True,
        "reveal": False,
        "delegation": False,
        "prize": False,
        "donation": False,
        "buy_tez": False,
        "sell_tez": False,
        "amount": o["contractBalance"] / 1e6,
        "fees": ((o["bakerFee"] + o["storageFee"] + o["allocationFee"]) / 1e6) if (o["status"] == "applied") else (o["bakerFee"] / 1e6),
        "tez_to_euros": o["quote"]["eur"],
        "tez_to_usd": o["quote"]["usd"],
        "token_id": None,
        "token_editions": None,
        "token_address": None,
        "hash": o["hash"],
        "comment": ""}

    # Add the processed origination
    originations.append(origination)

# Process the raw reveals
reveals = []

for r in raw_reveals:
    # Save the most relevant information
    reveal = {
        "timestamp": r["timestamp"],
        "level": r["level"],
        "kind": "reveal public key",
        "entrypoint": None,
        "parameters": None,
        "initiator": None,
        "sender": r["sender"]["address"],
        "target": None,
        "applied": r["status"] == "applied",
        "internal": False,
        "ignore": False,
        "mint": False,
        "collect": False,
        "active_offer": False,
        "art_sale": False,
        "collection_sale": False,
        "staking": False,
        "origination": False,
        "reveal": True,
        "delegation": False,
        "prize": False,
        "donation": False,
        "buy_tez": False,
        "sell_tez": False,
        "amount": 0,
        "fees": r["bakerFee"] / 1e6,
        "tez_to_euros": r["quote"]["eur"],
        "tez_to_usd": r["quote"]["usd"],
        "token_id": None,
        "token_editions": None,
        "token_address": None,
        "hash": r["hash"],
        "comment": ""}

    # Add the processed reveal
    reveals.append(reveal)

# Process the raw delegations
delegations = []

for d in raw_delegations:
    # Save the most relevant information
    delegation = {
        "timestamp": d["timestamp"],
        "level": d["level"],
        "kind": "delegation",
        "entrypoint": None,
        "parameters": None,
        "initiator": d["initiator"]["address"] if "initiator" in d else None,
        "sender": d["sender"]["address"],
        "target": None,
        "applied": d["status"] == "applied",
        "internal": False,
        "ignore": False,
        "mint": False,
        "collect": False,
        "active_offer": False,
        "art_sale": False,
        "collection_sale": False,
        "staking": False,
        "origination": False,
        "reveal": False,
        "delegation": True,
        "prize": False,
        "donation": False,
        "buy_tez": False,
        "sell_tez": False,
        "amount": 0,
        "fees": d["bakerFee"] / 1e6,
        "tez_to_euros": d["quote"]["eur"],
        "tez_to_usd": d["quote"]["usd"],
        "token_id": None,
        "token_editions": None,
        "token_address": None,
        "hash": r["hash"],
        "comment": ""}

    # Add the processed delegation
    delegations.append(delegation)

# Combine the transactions, originations and reveals in a single array
combined_operations = combine_operations(transactions, originations)
combined_operations = combine_operations(combined_operations, reveals)
combined_operations = combine_operations(combined_operations, delegations)

# Apply the operation corrections specified by the user
operation_corrections = read_json_file(os.path.join(data_directory, user_data, "operation_corrections.json"))

for operation in combined_operations:
    if operation["hash"] in operation_corrections:
        corrections = operation_corrections[operation["hash"]]

        for key, value in corrections.items():
            if key in operation:
                operation[key] = value

# Get all address aliases
aliases = {}
aliases.update(user_wallets)
aliases.update(baker_wallets)
aliases.update(exchange_wallets)
aliases.update(token_aliases)
aliases.update(smart_contract_aliases)
aliases.update({address: "Burn address" for address in burn_addresses})

for token_address, token_alias in fa12_tokens.items():
    if token_address not in aliases:
        aliases[token_address] = token_alias

for token_address, token_alias in fa2_tokens.items():
    if token_address not in aliases:
        aliases[token_address] = token_alias

# Save the processed data in a csv file
file_name = "operations_%s.csv" % user_first_wallet
columns = [
    "timestamp", "level", "tez_balance", "kind", "entrypoint", "initiator",
    "sender", "target", "is_initiator", "is_sender", "is_target", "applied",
    "internal", "ignore", "mint", "collect", "active_offer", "art_sale",
    "collection_sale", "staking", "origination", "reveal", "delegation",
    "prize", "donation", "buy_tez", "sell_tez", "amount", "fees",
    "received_amount", "art_sale_amount", "collection_sale_amount",
    "staking_rewards_amount", "prize_amount", "buy_tez_amount",
    "received_amount_others", "spent_amount", "collect_amount",
    "active_offer_amount", "donation_amount", "sell_tez_amount",
    "spent_amount_others", "spent_fees", "tez_to_euros", "tez_to_usd",
    "token_name", "token_id", "token_editions", "token_address", "tzkt_link",
    "comment"]
format = [
    "%s", "%i", "%f", "%s", "%s", "%s", "%s", "%s", "%r", "%r", "%r", "%r",
    "%r", "%r", "%r", "%r", "%r", "%r", "%r", "%r", "%r", "%r", "%r", "%r",
    "%r", "%r", "%r", "%f", "%f", "%f", "%f", "%f", "%f", "%f", "%f", "%f",
    "%f", "%f", "%f", "%f", "%f", "%f", "%f", "%f", "%f", "%s", "%s", "%s",
    "%s", "%s", "%s"]

with open(os.path.join(data_directory, "output", file_name), "w", newline="\n") as output_file:
    writer = csv.writer(output_file)

    # Write the header
    writer.writerow(columns)

    # Loop over the combined operations
    tez_balance = 0

    for op in combined_operations:
        # Check if the user is the initiator, the sender or the target
        is_initiator = op["initiator"] in user_wallets
        is_sender = op["sender"] in user_wallets
        is_target = op["target"] in user_wallets

        # Calculate the different received and spent tez amounts
        applied = op["applied"]
        ignore = op["ignore"]
        amount = op["amount"]
        fees = op["fees"]
        received_amount = amount if (is_target and applied and (not ignore)) else 0
        art_sale_amount = amount if (is_target and applied and (not ignore) and op["art_sale"]) else 0
        collection_sale_amount = amount if (is_target and applied and (not ignore) and op["collection_sale"]) else 0
        staking_rewards_amount = amount if (is_target and applied and (not ignore) and op["staking"]) else 0
        prize_amount = amount if (is_target and applied and (not ignore) and op["prize"]) else 0
        buy_tez_amount = amount if (is_target and applied and (not ignore) and op["buy_tez"]) else 0
        received_amount_others = amount if (is_target and applied and (not ignore) and (not (op["art_sale"] or op["collection_sale"] or op["staking"] or op["prize"] or op["buy_tez"]))) else 0
        spent_amount = amount if (is_sender and applied and (not ignore) and (not op["active_offer"])) else 0
        collect_amount = amount if (is_sender and applied and (not ignore) and op["collect"]) else 0
        active_offer_amount = amount if (is_sender and applied and (not ignore) and op["active_offer"]) else 0
        donation_amount = amount if (is_sender and applied and (not ignore) and op["donation"]) else 0
        sell_tez_amount = amount if (is_sender and applied and (not ignore) and op["sell_tez"]) else 0
        spent_amount_others = amount if (is_sender and applied and (not ignore) and (not (op["collect"] or op["active_offer"] or op["donation"] or op["sell_tez"]))) else 0
        spent_fees = fees if (is_initiator or is_sender) else 0

        # Calculate the tez balance
        tez_balance += received_amount - spent_amount - active_offer_amount - spent_fees

        # Get the token alias
        token_address = op["token_address"]
        token_alias = ""

        if token_address in token_aliases:
            token_alias = token_aliases[token_address]
        elif token_address in objktcom_collections:
            token_alias = "objkt.com collection"
        elif token_address in aliases:
            token_alias = aliases[token_address]

        token_alias = token_alias.replace(",", " ")

        # Write the operation data in the output file
        data = [
            op["timestamp"],
            op["level"],
            tez_balance,
            op["kind"],
            op["entrypoint"] if op["entrypoint"] is not None else "",
            aliases.get(op["initiator"], op["initiator"]).replace(",", " ") if op["initiator"] is not None else "",
            aliases.get(op["sender"], op["sender"]).replace(",", " ") if op["sender"] is not None else "",
            aliases.get(op["target"], op["target"]).replace(",", " ") if op["target"] is not None else "",
            is_initiator,
            is_sender,
            is_target,
            applied,
            op["internal"],
            ignore,
            op["mint"],
            op["collect"],
            op["active_offer"],
            op["art_sale"],
            op["collection_sale"],
            op["staking"],
            op["origination"],
            op["reveal"],
            op["delegation"],
            op["prize"],
            op["donation"],
            op["buy_tez"],
            op["sell_tez"],
            amount,
            fees,
            received_amount,
            art_sale_amount,
            collection_sale_amount,
            staking_rewards_amount,
            prize_amount,
            buy_tez_amount,
            received_amount_others,
            spent_amount,
            collect_amount,
            active_offer_amount,
            donation_amount,
            sell_tez_amount,
            spent_amount_others,
            spent_fees,
            op["tez_to_euros"],
            op["tez_to_usd"],
            token_alias,
            op["token_id"] if op["token_id"] is not None else "",
            op["token_editions"] if op["token_editions"] is not None else "",
            token_address if token_address is not None else "",
            "https://tzkt.io/%s" % op["hash"],
            op["comment"]]
        writer.writerow(data)

# Save the unprocessed transactions in a json file
unprocessed_file_name = "unprocessed_transactions_%s.json" % user_first_wallet
save_json_file(os.path.join(data_directory, "output", unprocessed_file_name), unprocessed_transactions)

# Print some details
print("\n We discovered %i operations associated to the user wallets." % len(combined_operations))
print(" You can find the processed information in %s\n" % os.path.join(data_directory, "output", file_name))

if len(unprocessed_transactions) > 0:
    print(" Of those, %i operations could not be processed completely." % len(unprocessed_transactions))
    print(" See %s for more details.\n" % os.path.join(data_directory, "output", unprocessed_file_name))


In [ ]:
# Get the raw token transfers associated with the user
raw_token_transfers = get_user_token_transfers(user_wallets)

# Process the raw token transfers
token_transfers = []

for tt in raw_token_transfers:
    # Save the most relevant information
    token_transfer = {
        "timestamp": tt["timestamp"],
        "level": tt["level"],
        "from": tt["from"]["address"] if "from" in tt else None,
        "to": tt["to"]["address"] if "to" in tt else None,
        "internal": False,
        "mint": False,
        "send": False,
        "receive": False,
        "burn": False,
        "token_id": tt["token"]["tokenId"],
        "token_editions": int(tt["amount"]),
        "token_address": tt["token"]["contract"]["address"],
        "token_standard": tt["token"]["standard"]}

    # Check if it's an internal transaction between the user wallets
    token_transfer["internal"] = (token_transfer["from"] in user_wallets) and (token_transfer["to"] in user_wallets)

    # Check if it's associated with a token mint
    token_transfer["mint"] = token_transfer["from"] is None

    # Check if the token was sent, received or burned
    token_transfer["send"] = (not token_transfer["internal"]) and (token_transfer["from"] in user_wallets)
    token_transfer["receive"] = (not token_transfer["internal"]) and (token_transfer["to"] in user_wallets)
    token_transfer["burn"] = (token_transfer["to"] is None) or (token_transfer["to"] in burn_addresses)

    # Add the processed token transfer
    token_transfers.append(token_transfer)

# Get all address aliases
aliases = {}
aliases.update(user_wallets)
aliases.update(token_aliases)
aliases.update(smart_contract_aliases)
aliases.update({address: "Burn address" for address in burn_addresses})

for token_address, token_alias in fa12_tokens.items():
    if token_address not in aliases:
        aliases[token_address] = token_alias

for token_address, token_alias in fa2_tokens.items():
    if token_address not in aliases:
        aliases[token_address] = token_alias

# Save the processed data in a csv file
file_name = "token_transfers_%s.csv" % user_first_wallet
columns = [
    "timestamp", "level", "from", "to", "internal", "mint", "send", "receive",
    "burn", "token_name", "token_id", "token_editions", "token_address",
    "token_standard", "token_link"]
format = [
    "%s", "%i", "%s", "%s", "%r", "%r", "%r", "%r", "%r", "%s", "%s", "%s",
    "%s", "%s", "%s"]

with open(os.path.join(data_directory, "output", file_name), "w", newline="\n") as output_file:
    writer = csv.writer(output_file)

    # Write the header
    writer.writerow(columns)

    # Loop over the token transfers
    for tt in token_transfers:
        # Get the token alias
        token_id = tt["token_id"]
        token_address = tt["token_address"]
        token_alias = ""

        if token_address in token_aliases:
            token_alias = token_aliases[token_address]
        elif token_address in objktcom_collections:
            token_alias = "objkt.com collection"
        elif token_address in aliases:
            token_alias = aliases[token_address]

        token_alias = token_alias.replace(",", " ")

        # Write the token transfer data in the output file
        data = [
            tt["timestamp"],
            tt["level"],
            aliases.get(tt["from"], tt["from"]).replace(",", " ") if tt["from"] is not None else "",
            aliases.get(tt["to"], tt["to"]).replace(",", " ") if tt["to"] is not None else "",
            tt["internal"],
            tt["mint"],
            tt["send"],
            tt["receive"],
            tt["burn"],
            token_alias,
            token_id if token_id is not None else "",
            tt["token_editions"] if tt["token_editions"] is not None else "",
            token_address if token_address is not None else "",
            tt["token_standard"],
            get_token_link(token_alias, token_id, token_address)]
        writer.writerow(data)

# Print some details
print("\n We discovered %i token transfers associated to the user wallets." % len(token_transfers))
print(" You can find the processed information in %s\n" % os.path.join(data_directory, "output", file_name))

In [ ]:
user_first_wallet = list(user_wallets.keys())[0]

# Load the csv file containing all the user operations
file_name = "operations_%s.csv" % user_first_wallet
operations = pd.read_csv(os.path.join(data_directory, "output", file_name), parse_dates=["timestamp"], keep_default_na=False)

# Load the csv file containing all the user token transfers
file_name = "token_transfers_%s.csv" % user_first_wallet
token_transfers = pd.read_csv(os.path.join(data_directory, "output", file_name), parse_dates=["timestamp"], keep_default_na=False)

# Change the token editions column type to str
token_transfers["token_id"] = token_transfers["token_id"].astype(str)
token_transfers["token_editions"] = token_transfers["token_editions"].astype(str)

# Add a column to indicate which transfers have been used during the next steps
token_transfers["used"] = False

# Don't use internal token transfers
token_transfers.loc[token_transfers["internal"], "used"] = True

# Get the information about the minted, collected, dropped, sold and transferred tokens
minted_tokens = get_minted_tokens(operations, token_transfers)
collected_tokens = get_collected_tokens(operations, token_transfers)
dropped_tokens = get_dropped_tokens(operations, token_transfers, collected_tokens)
sold_tokens = get_sold_tokens(operations, token_transfers)
transferred_tokens = get_transferred_tokens(operations, token_transfers)

# Save the data frames as csv files
minted_tokens_file_name = "minted_tokens_%s.csv" % user_first_wallet
minted_tokens.to_csv(os.path.join(data_directory, "output", minted_tokens_file_name), index=False)
collected_tokens_file_name = "collected_tokens_%s.csv" % user_first_wallet
collected_tokens.to_csv(os.path.join(data_directory, "output", collected_tokens_file_name), index=False)
dropped_tokens_file_name = "dropped_tokens_%s.csv" % user_first_wallet
dropped_tokens.to_csv(os.path.join(data_directory, "output", dropped_tokens_file_name), index=False)
sold_tokens_file_name = "sold_tokens_%s.csv" % user_first_wallet
sold_tokens.to_csv(os.path.join(data_directory, "output", sold_tokens_file_name), index=False)
transferred_tokens_file_name = "transferred_tokens_%s.csv" % user_first_wallet
transferred_tokens.to_csv(os.path.join(data_directory, "output", transferred_tokens_file_name), index=False)

# Print some details
print("\n NFT operations associated to the user wallets:\n")

if minted_tokens.size > 0:
    print("  - Minted %i tokens." % (minted_tokens["token_id"] + minted_tokens["token_address"]).unique().size)
    print("    See %s for more details.\n" % os.path.join(data_directory, "output", minted_tokens_file_name))

if collected_tokens.size > 0:
    print("  - Collected %i tokens from other artists." % (collected_tokens["token_id"] + collected_tokens["token_address"]).unique().size)
    print("    See %s for more details.\n" % os.path.join(data_directory, "output", collected_tokens_file_name))

if dropped_tokens.size > 0:
    print("  - Received %i tokens as free drops." % (dropped_tokens["token_id"] + dropped_tokens["token_address"]).unique().size)
    print("    See %s for more details.\n" % os.path.join(data_directory, "output", dropped_tokens_file_name))

if sold_tokens.size > 0:
    print("  - Sold %i tokens collected from other artists." % (sold_tokens["token_id"] + sold_tokens["token_address"]).unique().size)
    print("    See %s for more details.\n" % os.path.join(data_directory, "output", sold_tokens_file_name))

if transferred_tokens.size > 0:
    print("  - Burned or transferred to other users %i tokens." % (transferred_tokens["token_id"] + transferred_tokens["token_address"]).unique().size)
    print("    See %s for more details.\n" % os.path.join(data_directory, "output", transferred_tokens_file_name))


In [ ]:
# Change the token_id column data type to string
minted_tokens = minted_tokens.astype({"token_id": str})
collected_tokens = collected_tokens.astype({"token_id": str})
dropped_tokens = dropped_tokens.astype({"token_id": str})
sold_tokens = sold_tokens.astype({"token_id": str})
transferred_tokens = transferred_tokens.astype({"token_id": str})

# Load the tax parameters provided by the user
tax_parameters = read_json_file(os.path.join(data_directory, user_data, "tax_parameters.json"))
start_date = pd.to_datetime(tax_parameters["start_date"])
end_date = pd.to_datetime(tax_parameters["end_date"])
hold_period = 365 * tax_parameters["hold_period_in_years"]
fiat_coin = "euros" if tax_parameters["fiat_coin"] == "eur" else "USD"

# Get the token trades information
token_trades = get_token_trades(collected_tokens, dropped_tokens, sold_tokens, transferred_tokens, hold_period)

# Save the token trades as a csv file
token_trades_file_name = "token_trades_%s.csv" % user_first_wallet
token_trades.to_csv(os.path.join(data_directory, "output", token_trades_file_name), index=False)

# Get the tez exchange gains associated to collect and tez exchange operations
tez_exhange_gains = get_tez_exchange_gains(operations, hold_period)

# Select the data that should be considered for the tax calculations
cond = (operations["timestamp"] >= start_date) & (operations["timestamp"] <= end_date)
operations = operations[cond]
cond = (minted_tokens["timestamp"] >= start_date) & (minted_tokens["timestamp"] <= end_date)
minted_tokens = minted_tokens[cond]
cond = (collected_tokens["timestamp"] >= start_date) & (collected_tokens["timestamp"] <= end_date)
collected_tokens = collected_tokens[cond]
cond = (dropped_tokens["timestamp"] >= start_date) & (dropped_tokens["timestamp"] <= end_date)
dropped_tokens = dropped_tokens[cond]
cond = (sold_tokens["timestamp"] >= start_date) & (sold_tokens["timestamp"] <= end_date)
sold_tokens = sold_tokens[cond]
cond = (transferred_tokens["timestamp"] >= start_date) & (transferred_tokens["timestamp"] <= end_date)
transferred_tokens = transferred_tokens[cond]
cond = (token_trades["sell_timestamp"] >= start_date) & (token_trades["sell_timestamp"] <= end_date)
token_trades = token_trades[cond]
cond = (tez_exhange_gains["timestamp"] >= start_date) & (tez_exhange_gains["timestamp"] <= end_date)
tez_exhange_gains = tez_exhange_gains[cond]

# Use the correct fiat columns
if fiat_coin == "euros":
    tez_to_fiat = operations["tez_to_euros"]
    token_trades_gain_fiat = token_trades["taxed_gain_euros"]
    tez_exchange_gain_fiat = tez_exhange_gains["taxed_gain_euros"]
else:
    tez_to_fiat = operations["tez_to_usd"]
    token_trades_gain_fiat = token_trades["taxed_gain_usd"]
    tez_exchange_gain_fiat = tez_exhange_gains["taxed_gain_usd"]

# Print some details
print("\n Summary for the period between %s and %s." % (start_date, end_date))
print(" %i operations were found for that period." % len(operations))

print("\n Total amount of tez received by the user wallets: %.2f tez (%.2f %s).\n" % (
    operations["received_amount"].sum(), (operations["received_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) bought in exchanges." % (
    operations["buy_tez_amount"].sum(), (operations["buy_tez_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) came from sales of the user minted art." % (
    operations["art_sale_amount"].sum(), (operations["art_sale_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) came from reselling NFTs minted by other artists." % (
    operations["collection_sale_amount"].sum(), (operations["collection_sale_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) came from prizes (hackathons, community rewards, etc)." % (
    operations["prize_amount"].sum(), (operations["prize_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) came from staking rewards." % (
    operations["staking_rewards_amount"].sum(), (operations["staking_rewards_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) came from other sources." % (
    operations["received_amount_others"].sum(), (operations["received_amount_others"] * tez_to_fiat).sum(), fiat_coin))

print("\n Total amount of tez spent with the user wallets: %.2f tez (%.2f %s).\n" % (
    (operations["spent_amount"] + operations["active_offer_amount"] + operations["spent_fees"]).sum(),
    ((operations["spent_amount"] + operations["active_offer_amount"] + operations["spent_fees"]) * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) sold in exchanges." % (
    operations["sell_tez_amount"].sum(), (operations["sell_tez_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) were spent collecting other artists art." % (
    operations["collect_amount"].sum(), (operations["collect_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) are still in active offers to collect other artists art." % (
    operations["active_offer_amount"].sum(), (operations["active_offer_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) were spent in donations." % (
    operations["donation_amount"].sum(), (operations["donation_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) were spent in transaction fees." % (
    operations["spent_fees"].sum(), (operations["spent_fees"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) were spent in other uses." % (
    operations["spent_amount_others"].sum(), (operations["spent_amount_others"] * tez_to_fiat).sum(), fiat_coin))

print("\n NFT operations associated to the user wallets:\n")

if minted_tokens.size > 0:
    print("   Minted %i tokens." % (minted_tokens["token_id"] + minted_tokens["token_address"]).unique().size)

if collected_tokens.size > 0:
    print("   Collected %i tokens from other artists." % (collected_tokens["token_id"] + collected_tokens["token_address"]).unique().size)

if dropped_tokens.size > 0:
    print("   Received %i tokens as free drops." % (dropped_tokens["token_id"] + dropped_tokens["token_address"]).unique().size)

if sold_tokens.size > 0:
    print("   Sold %i tokens collected from other artists." % (sold_tokens["token_id"] + sold_tokens["token_address"]).unique().size)

if transferred_tokens.size > 0:
    print("   Burned or transferred to other users %i tokens." % (transferred_tokens["token_id"] + transferred_tokens["token_address"]).unique().size)

print("\n Taxable gains:\n")
print("   %.2f tez (%.2f %s) from sales of the user minted art." % (
    operations["art_sale_amount"].sum(), (operations["art_sale_amount"] * tez_to_fiat).sum(), fiat_coin))

if token_trades_gain_fiat.sum() > 0:
    print("   %.2f tez (%.2f %s) from trades of NFTs minted by other artists." % (
        token_trades["taxed_gain"].sum(), token_trades_gain_fiat.sum(), fiat_coin))

print("   %.2f tez (%.2f %s) from prizes (hackathons, community rewards, etc)." % (
    operations["prize_amount"].sum(), (operations["prize_amount"] * tez_to_fiat).sum(), fiat_coin))
print("   %.2f tez (%.2f %s) from staking rewards." % (
    operations["staking_rewards_amount"].sum(), (operations["staking_rewards_amount"] * tez_to_fiat).sum(), fiat_coin))

if tez_exchange_gain_fiat.sum() > 0:
    print("   %.2f %s from tez exchange gains associated to tez exchange operations (using the FIFO method)." % (
        tez_exchange_gain_fiat.sum(), fiat_coin))

print("\n Possible tax-deductible losses:\n")
print("   %.2f tez (%.2f %s) from tezos transaction fees." % (
    operations["spent_fees"].sum(), (operations["spent_fees"] * tez_to_fiat).sum(), fiat_coin))

if operations["donation_amount"].sum() > 0:
    print("   %.2f tez (%.2f %s) from donations." % (
        operations["donation_amount"].sum(), (operations["donation_amount"] * tez_to_fiat).sum(), fiat_coin))

if token_trades_gain_fiat.sum() < 0:
    print("   %.2f tez (%.2f %s) from trades of NFTs minted by other artists." % (
        abs(token_trades["gain"].sum()), abs(token_trades_gain_fiat.sum()), fiat_coin))

if tez_exchange_gain_fiat.sum() < 0:
    print("   %.2f %s from tez exchange losses associated to tez exchange operations (using the FIFO method)." % (
        abs(tez_exchange_gain_fiat.sum()), fiat_coin))

print("\n You can find the token trades information in %s\n" % os.path.join(data_directory, "output", token_trades_file_name))



In [ ]:
ops = pd.DataFrame(combined_operations)

for wallet in user_wallets:
    print(wallet)
    koinly = get_koinly_csv(ops, wallet)
    file_name = "koinly_%s.join" % wallet
    save_json_file(os.path.join(data_directory, "output", file_name), koinly)
    break # todo remove

In [ ]:
ops = pd.DataFrame(combined_operations)
targets = ops.loc[ops['target'] == 'tz1cTS1WwovU7SC783xgJxZrzr151mcshmNi']
senders = ops.loc[ops['sender'] == 'tz1cTS1WwovU7SC783xgJxZrzr151mcshmNi']
print(len(targets))
print(len(senders))
main = pd.concat([targets, senders])
print(len(main))
print(main)